# <h1 align="center" id="heading">Ichimoku Clouds Speculations</h1>

In this notebook, we'll be backtesting **Ichimoku Clouds** indicator in many markets and different ways.

I'll not explain what is or its lore so if you're interested visit [Ichimoku Kinkō Hyō](https://en.wikipedia.org/wiki/Ichimoku_Kinkō_Hyō).

As someone said **"Don't reinvent the wheel!"**, I will use some functions developed by other people. One of them is **Derek Banas**, who on his [Youtube channel](https://www.youtube.com/c/derekbanas) teaches how to analyze investment strategies using algorithms and many other cool things.


## First things first

So what's needed to start our speculations?

We will test Ichimoku's Cloud in the following investment ways:

* Forex Market
* Crypto Market
* Stock Market

Well, we will need the following:

1. Collect some ticker data from any source.
2. Write a function to add Ichimoku's Cloud to our tickers.
3. Write a function for each way to use Ichimoku's Cloud.
4. Back test our functions with some parameters.

### Before everything, lets import the modules that we will use.

In [47]:
# Importing all the important modules that we're gonna use

import numpy as np
import pandas as pd
import csv
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "notebook_connected"

### 1. Lets collect some tickers

We will work with 1hour candles and 1day candles.

Next we'll select 3 pairs of each market listed above divided into 2 timestamps to test with Ichimoku's Cloud.

All the tickers was downloaded from [Forex Historical Data App](https://eatradingacademy.com/software/forex-historical-data/).


In [36]:
# Function to read the csv file and convert to a dataframe
def csv_to_df(fileName):
    
    # Open the file
    with open(fileName, newline='') as csvfile:
    
        Timestamp = []
        Open = []
        High = []
        Low = []
        Close = []
        Volume = []
        
        # Read the csv file and parse it with a "tab" delimeter
        csvText = csv.reader(csvfile, delimiter='	', quotechar='|')
        
        # Iterate through the data and push the row information to our arrays
        for row in csvText:
            Timestamp.append(str(row[0]))
            Open.append(float(row[1]))
            High.append(float(row[2]))
            Low.append(float(row[3]))
            Close.append(float(row[4]))
            Volume.append(int(row[5]))
        
        # Get the arrays and build the dataframe
        df = pd.DataFrame({ 'Open': Open, 'High': High, 'Low': Low, 'Close': Close, 'Volume': Volume, "Timestamp": Timestamp })
        
        return df

# Creating all the dataframes we will need

# Forex 1h
eurusd_h = csv_to_df('EURUSD60.csv')
gbpjpy_h = csv_to_df('GBPJPY60.csv')
xauusd_h = csv_to_df('XAUUSD60.csv')
# Forex 1d
eurusd_d = csv_to_df('EURUSD1440.csv')
gbpjpy_d = csv_to_df('GBPJPY1440.csv')
xauusd_d = csv_to_df('XAUUSD1440.csv')

# Crypto 1h
btcusd_h = csv_to_df('BTCUSD60.csv')
ethusd_h = csv_to_df('ETHUSD60.csv')
adausd_h = csv_to_df('ADAUSDT60.csv')
# Crypto 1d
btcusd_d = csv_to_df('BTCUSD1440.csv')
ethusd_d = csv_to_df('ETHUSD1440.csv')
adausd_d = csv_to_df('ADAUSDT1440.csv')

# Stocks 1h
aapl_h = csv_to_df('AAPLUSUSD60.csv')
nflx_h = csv_to_df('NFLXUSUSD60.csv')
tsla_h = csv_to_df('TSLAUSUSD60.csv')

# Stocks 1d
aapl_d = csv_to_df('AAPLUSUSD1440.csv')
nflx_d = csv_to_df('NFLXUSUSD1440.csv')
tsla_d = csv_to_df('TSLAUSUSD1440.csv')

### 2. Writing our Ichimoku's Cloud function

The Ichimoku's Cloud indicator is a group of 5 lines:

1. **Tenkan-sen**: (Conversion Line): (9-period high + 9-period low)/2))
2. **Kijun-sen**: (Base Line): (26-period high + 26-period low)/2))
3. **Senkou Span A**: (Leading Span A): (Conversion Line + Base Line)/2))
4. **Senkou Span B**: (Leading Span B): (52-period high + 52-period low)/2))
5. **Chikou Span**: (Lagging Span): Close plotted 26 periods in the past

In [37]:
# Function to edit the dataframe and add the Ichimoku key values
def add_Ichimoku_to_df(df):

    # Conversion
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + low_val) / 2

    # Baseline
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + low_val2) / 2

    # Span A
    df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2).shift(26)

    # Span B
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)

    # Lagging Span
    df['Lagging'] = df['Close'].shift(-26)

    return df

#### Add Ichimoku's Cloud to our existing dataframes

In [38]:
# Forex 1h
eurusd_h_ichimoku = add_Ichimoku_to_df(eurusd_h)
gbpjpy_h_ichimoku = add_Ichimoku_to_df(gbpjpy_h)
xauusd_h_ichimoku = add_Ichimoku_to_df(xauusd_h)
# Forex 1d
eurusd_d_ichimoku = add_Ichimoku_to_df(eurusd_d)
gbpjpy_d_ichimoku = add_Ichimoku_to_df(gbpjpy_d)
xauusd_d_ichimoku = add_Ichimoku_to_df(xauusd_d)

# Crypto 1h
btcusd_h_ichimoku = add_Ichimoku_to_df(btcusd_h)
ethusd_h_ichimoku = add_Ichimoku_to_df(ethusd_h)
adausd_h_ichimoku = add_Ichimoku_to_df(adausd_h)
# Crypto 1d
btcusd_d_ichimoku = add_Ichimoku_to_df(btcusd_d)
ethusd_d_ichimoku = add_Ichimoku_to_df(ethusd_d)
adausd_d_ichimoku = add_Ichimoku_to_df(adausd_d)

# Stocks 1h
aapl_h_ichimoku = add_Ichimoku_to_df(aapl_h)
nflx_h_ichimoku = add_Ichimoku_to_df(nflx_h)
tsla_h_ichimoku = add_Ichimoku_to_df(tsla_h)
# Stocks 1d
aapl_d_ichimoku = add_Ichimoku_to_df(aapl_d)
nflx_d_ichimoku = add_Ichimoku_to_df(nflx_d)
tsla_d_ichimoku = add_Ichimoku_to_df(tsla_d)

### 3. Now, we create a function for each way to use Ichimoku's Cloud

Based on a study made by [Luís António Gomes Almeida](https://revistas.unal.edu.co/index.php/innovar/article/view/99677), Ichimoku's Cloud has different ways to use it. Here are some of them:

* **T vs K** - Tenkan-sen versus Kijun-sen.

        BUY/LONG: Tenkan-sen > Kijun-sen.
        SELL/SHORT: Tenkan-sen < Kijun-sen.
        
* **T vs K vs P** - Tenkan-sen versus Kijun-sen versus Price.
        
        BUY/LONG: Price > Tenkan-sen > Kijun-sen.
        SELL/SHORT: Price < Tenkan-sen < Kijun-sen.

* **Ch vs P** - Chikou-span versus Price.

        BUY/LONG: Chikou-span > Price.
        SELL/SHORT: Chikou-span < Price.

* **Ch vs P vs C** - Chikou-span versus Price versus Cloud.

        BUY/LONG: Chikou-span > Price > Senkou-span A > Senkou-span B.
        SELL/SHORT: Chikou-span < Price < Senkou-span A < Senkou-span B.

* **5 lines and Cloud** - All the 5 lines and the Cloud.

        BUY/LONG: Chikou-span > Price > Tenkan-sen > Kijun-sen > Senkou-span A > Senkou-span B.
        SELL/SHORT: Chikou-span < Price < Tenkan-sen < Kijun-sen < Senkou-span A < Senkou-span B.


This 5 examples shows how traders uses Ichimoku Clouds within their operations.
Keep in mind that some of the functions listed are speculative and for testing purposes only as they have an analysis of 26 periods in the future and thus impossible to make entries when the conditions are met. Therefore, as it makes no sense to keep an imaginary result in our calculations, the functions that involve **Chikou-span** were adjusted with the 26 periods to be able to obtain "real" results.

### T_K

In [ ]:
def t_k(df):
    
    df['label'] = np.where(df['Conversion'] > df['Baseline'], 1, 0)
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()
    print(len(df))

    df = df.groupby('group')
    
    result = []
    
    for i, group in df:
        result.append({
            'operation': 'buy' if group.iloc[0]['label'] == 1 else 'sell',
            'startTime': group.iloc[0]['Timestamp'],
            'startValue': group.iloc[0]['Close'],
            'endTime': group.iloc[-1]['Timestamp'],
            'endValue': group.iloc[-1]['Close'],
            "result": (group.iloc[-1]['Close'] - group.iloc[0]['Close']) if group.iloc[0]['label'] == 1 else (group.iloc[0]['Close'] - group.iloc[-1]['Close'])
        })
    
    return result

t_k_eurusd_h = t_k(eurusd_h_ichimoku)
t_k_gbpjpy_h = t_k(gbpjpy_h_ichimoku)
t_k_xauusd_h = t_k(xauusd_h_ichimoku)
t_k_eurusd_d = t_k(eurusd_d_ichimoku)
t_k_gbpjpy_d = t_k(gbpjpy_d_ichimoku)
t_k_xauusd_d = t_k(xauusd_d_ichimoku)

t_k_btcusd_h = t_k(btcusd_h_ichimoku)
t_k_ethusd_h = t_k(ethusd_h_ichimoku)
t_k_adausd_h = t_k(adausd_h_ichimoku)
t_k_btcusd_d = t_k(btcusd_d_ichimoku)
t_k_ethusd_d = t_k(ethusd_d_ichimoku)
t_k_adausd_d = t_k(adausd_d_ichimoku)

t_k_aapl_h = t_k(aapl_h_ichimoku)
t_k_nflx_h = t_k(nflx_h_ichimoku)
t_k_tsla_h = t_k(tsla_h_ichimoku)
t_k_aapl_d = t_k(aapl_d_ichimoku)
t_k_nflx_d = t_k(nflx_d_ichimoku)
t_k_tsla_d = t_k(tsla_d_ichimoku)

### T_K_P

In [40]:
def t_k_p(df):
    
    labels = []
    
    lastLabel = 0
    
    for i, row in df.iterrows():
        
        if row['Close'] > row['Conversion'] and row['Conversion'] > row['Baseline']:
            labels.append(1)
            lastLabel = 1
        elif row['Close'] < row['Conversion'] and row['Conversion'] < row['Baseline']:
            labels.append(0)
            lastLabel = 0
        else:
            labels.append(lastLabel)
    
    df['label'] = labels
    
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()

    df = df.groupby('group')
    
    result = []
    
    for i, group in df:
        result.append({
            'operation': 'buy' if group.iloc[0]['label'] == 1 else 'sell',
            'startTime': group.iloc[0]['Timestamp'],
            'startValue': group.iloc[0]['Close'],
            'endTime': group.iloc[-1]['Timestamp'],
            'endValue': group.iloc[-1]['Close'],
            "result": (group.iloc[-1]['Close'] - group.iloc[0]['Close']) if group.iloc[0]['label'] == 1 else (group.iloc[0]['Close'] - group.iloc[-1]['Close'])
        })
    
    return result

t_k_p_eurusd_h = t_k_p(eurusd_h_ichimoku)
t_k_p_gbpjpy_h = t_k_p(gbpjpy_h_ichimoku)
t_k_p_xauusd_h = t_k_p(xauusd_h_ichimoku)
t_k_p_eurusd_d = t_k_p(eurusd_d_ichimoku)
t_k_p_gbpjpy_d = t_k_p(gbpjpy_d_ichimoku)
t_k_p_xauusd_d = t_k_p(xauusd_d_ichimoku)

t_k_p_btcusd_h = t_k_p(btcusd_h_ichimoku)
t_k_p_ethusd_h = t_k_p(ethusd_h_ichimoku)
t_k_p_adausd_h = t_k_p(adausd_h_ichimoku)
t_k_p_btcusd_d = t_k_p(btcusd_d_ichimoku)
t_k_p_ethusd_d = t_k_p(ethusd_d_ichimoku)
t_k_p_adausd_d = t_k_p(adausd_d_ichimoku)

t_k_p_aapl_h = t_k_p(aapl_h_ichimoku)
t_k_p_nflx_h = t_k_p(nflx_h_ichimoku)
t_k_p_tsla_h = t_k_p(tsla_h_ichimoku)
t_k_p_aapl_d = t_k_p(aapl_d_ichimoku)
t_k_p_nflx_d = t_k_p(nflx_d_ichimoku)
t_k_p_tsla_d = t_k_p(tsla_d_ichimoku)

### Ch_P

In [41]:
def ch_p(df):
    
    ndf = df.copy()
    
    labels = []
    
    lastLabel = 0
    
    for i, candle in df.iterrows():
        
        if i > 26 and i < len(ndf):
            
            if ndf.iloc[i-26]['Lagging'] > ndf.iloc[i-26]['High']:
                labels.append(1)
                lastLabel = 1
            elif ndf.iloc[i-26]['Lagging'] < ndf.iloc[i-26]['Low']:
                labels.append(0)
                lastLabel = 0
            else:
                labels.append(lastLabel)
        else:
            labels.append(0)
    
    ndf['label'] = labels
    
    ndf['group'] = ndf['label'].ne(ndf['label'].shift()).cumsum()

    ndf = ndf.groupby('group')
    
    result = []
    
    for index, group in ndf:
        result.append({
            'operation': 'buy' if group.iloc[0]['label'] == 1 else 'sell',
            'startTime': group.iloc[0]['Timestamp'],
            'startValue': group.iloc[0]['Close'],
            'endTime': group.iloc[-1]['Timestamp'],
            'endValue': group.iloc[-1]['Close'],
            "result": (group.iloc[-1]['Close'] - group.iloc[0]['Close']) if group.iloc[0]['label'] == 1 else (group.iloc[0]['Close'] - group.iloc[-1]['Close'])
        })
    
    return result

ch_p_eurusd_h = ch_p(eurusd_h_ichimoku)
ch_p_gbpjpy_h = ch_p(gbpjpy_h_ichimoku)
ch_p_xauusd_h = ch_p(xauusd_h_ichimoku)
ch_p_eurusd_d = ch_p(eurusd_d_ichimoku)
ch_p_gbpjpy_d = ch_p(gbpjpy_d_ichimoku)
ch_p_xauusd_d = ch_p(xauusd_d_ichimoku)

ch_p_btcusd_h = ch_p(btcusd_h_ichimoku)
ch_p_ethusd_h = ch_p(ethusd_h_ichimoku)
ch_p_adausd_h = ch_p(adausd_h_ichimoku)
ch_p_btcusd_d = ch_p(btcusd_d_ichimoku)
ch_p_ethusd_d = ch_p(ethusd_d_ichimoku)
ch_p_adausd_d = ch_p(adausd_d_ichimoku)

ch_p_aapl_h = ch_p(aapl_h_ichimoku)
ch_p_nflx_h = ch_p(nflx_h_ichimoku)
ch_p_tsla_h = ch_p(tsla_h_ichimoku)
ch_p_aapl_d = ch_p(aapl_d_ichimoku)
ch_p_nflx_d = ch_p(nflx_d_ichimoku)
ch_p_tsla_d = ch_p(tsla_d_ichimoku)

### Ch_P_C

In [42]:
def ch_p_c(df):
    
    ndf = df.copy()
    
    labels = []
    
    lastLabel = 0
    
    for i, row in ndf.iterrows():
        
        if i > 26 and i < len(ndf):
        
            if ndf.iloc[i-26]['Lagging'] > ndf.iloc[i-26]['Close'] and row['Close'] > row['SpanA'] and row['SpanA'] > row['SpanB']:
                labels.append(1)
                lastLabel = 1
            elif ndf.iloc[i-26]['Lagging'] < ndf.iloc[i-26]['Close'] and row['Close'] < row['SpanA'] and row['SpanA'] < row['SpanB']:
                labels.append(0)
                lastLabel = 0
            else:
                labels.append(lastLabel)
        else:
            labels.append(0)
    
    ndf['label'] = labels
    
    ndf['group'] = ndf['label'].ne(ndf['label'].shift()).cumsum()

    ndf = ndf.groupby('group')
    
    result = []
    
    for i, group in ndf:
        result.append({
            'operation': 'buy' if group.iloc[0]['label'] == 1 else 'sell',
            'startTime': group.iloc[0]['Timestamp'],
            'startValue': group.iloc[0]['Close'],
            'endTime': group.iloc[-1]['Timestamp'],
            'endValue': group.iloc[-1]['Close'],
            "result": (group.iloc[-1]['Close'] - group.iloc[0]['Close']) if group.iloc[0]['label'] == 1 else (group.iloc[0]['Close'] - group.iloc[-1]['Close'])
        })
    
    return result

ch_p_c_eurusd_h = ch_p_c(eurusd_h_ichimoku)
ch_p_c_gbpjpy_h = ch_p_c(gbpjpy_h_ichimoku)
ch_p_c_xauusd_h = ch_p_c(xauusd_h_ichimoku)
ch_p_c_eurusd_d = ch_p_c(eurusd_d_ichimoku)
ch_p_c_gbpjpy_d = ch_p_c(gbpjpy_d_ichimoku)
ch_p_c_xauusd_d = ch_p_c(xauusd_d_ichimoku)

ch_p_c_btcusd_h = ch_p_c(btcusd_h_ichimoku)
ch_p_c_ethusd_h = ch_p_c(ethusd_h_ichimoku)
ch_p_c_adausd_h = ch_p_c(adausd_h_ichimoku)
ch_p_c_btcusd_d = ch_p_c(btcusd_d_ichimoku)
ch_p_c_ethusd_d = ch_p_c(ethusd_d_ichimoku)
ch_p_c_adausd_d = ch_p_c(adausd_d_ichimoku)

ch_p_c_aapl_h = ch_p_c(aapl_h_ichimoku)
ch_p_c_nflx_h = ch_p_c(nflx_h_ichimoku)
ch_p_c_tsla_h = ch_p_c(tsla_h_ichimoku)
ch_p_c_aapl_d = ch_p_c(aapl_d_ichimoku)
ch_p_c_nflx_d = ch_p_c(nflx_d_ichimoku)
ch_p_c_tsla_d = ch_p_c(tsla_d_ichimoku)

### 5 lines and cloud

In [43]:
def five_l_c(df):
    
    labels = []
    
    lastLabel = 0
    
    for i, row in df.iterrows():
        
        if row['Lagging'] > row['Close'] and row['Close'] > row['Conversion'] and row['Conversion'] > row['Baseline'] and row['Baseline'] > row['SpanA'] and row['SpanA'] > row['SpanB']:
            labels.append(1)
            lastLabel = 1
        elif row['Lagging'] < row['Close'] and row['Close'] < row['Conversion'] and row['Conversion'] < row['Baseline'] and row['Baseline'] < row['SpanA'] and row['SpanA'] < row['SpanB']:
            labels.append(0)
            lastLabel = 0
        else:
            labels.append(lastLabel)
    
    df['label'] = labels
    
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()

    df = df.groupby('group')
    
    result = []
    
    for i, group in df:
        result.append({
            'operation': 'buy' if group.iloc[0]['label'] == 1 else 'sell',
            'startTime': group.iloc[0]['Timestamp'],
            'startValue': group.iloc[0]['Close'],
            'endTime': group.iloc[-1]['Timestamp'],
            'endValue': group.iloc[-1]['Close'],
            "result": (group.iloc[-1]['Close'] - group.iloc[0]['Close']) if group.iloc[0]['label'] == 1 else (group.iloc[0]['Close'] - group.iloc[-1]['Close'])
        })
    
    return result

five_l_c_eurusd_h = five_l_c(eurusd_h_ichimoku)
five_l_c_gbpjpy_h = five_l_c(gbpjpy_h_ichimoku)
five_l_c_xauusd_h = five_l_c(xauusd_h_ichimoku)
five_l_c_eurusd_d = five_l_c(eurusd_d_ichimoku)
five_l_c_gbpjpy_d = five_l_c(gbpjpy_d_ichimoku)
five_l_c_xauusd_d = five_l_c(xauusd_d_ichimoku)

five_l_c_btcusd_h = five_l_c(btcusd_h_ichimoku)
five_l_c_ethusd_h = five_l_c(ethusd_h_ichimoku)
five_l_c_adausd_h = five_l_c(adausd_h_ichimoku)
five_l_c_btcusd_d = five_l_c(btcusd_d_ichimoku)
five_l_c_ethusd_d = five_l_c(ethusd_d_ichimoku)
five_l_c_adausd_d = five_l_c(adausd_d_ichimoku)

five_l_c_aapl_h = five_l_c(aapl_h_ichimoku)
five_l_c_nflx_h = five_l_c(nflx_h_ichimoku)
five_l_c_tsla_h = five_l_c(tsla_h_ichimoku)
five_l_c_aapl_d = five_l_c(aapl_d_ichimoku)
five_l_c_nflx_d = five_l_c(nflx_d_ichimoku)
five_l_c_tsla_d = five_l_c(tsla_d_ichimoku)

### 4. The speculations

Now, before we test the Ichimoku's Cloud using the functions we wrote, let's first visualize some values.

As some of you know, payouts are based on the price change between trades, so lets write some code to visualize the price change for our functions.

In [ ]:
def calculate_prices(strategies):
    
    result = []
    
    for strategy in strategies:
        price = 0
        for row in strategy:
            if row['operation']:
            
                if row['operation'] == 'sell':
                    price += row['startValue'] - row['endValue']
                else:
                    price += row['endValue'] - row['startValue']
                    
            result.append(price)
    
    return result

eurusd_h_prices = calculate_prices([t_k_eurusd_h, t_k_p_eurusd_h, ch_p_eurusd_h, ch_p_c_eurusd_h[0], five_l_c_eurusd_h])
gbpjpy_h_prices = calculate_prices([t_k_gbpjpy_h, t_k_p_gbpjpy_h, ch_p_gbpjpy_h, ch_p_c_gbpjpy_h[0], five_l_c_gbpjpy_h])
xauusd_h_prices = calculate_prices([t_k_xauusd_h, t_k_p_xauusd_h, ch_p_xauusd_h, ch_p_c_xauusd_h[0], five_l_c_xauusd_h])
eurusd_d_prices = calculate_prices([t_k_eurusd_d, t_k_p_eurusd_d, ch_p_eurusd_d, ch_p_c_eurusd_d[0], five_l_c_eurusd_d])
gbpjpy_d_prices = calculate_prices([t_k_gbpjpy_d, t_k_p_gbpjpy_d, ch_p_gbpjpy_d, ch_p_c_gbpjpy_d[0], five_l_c_gbpjpy_d])
xauusd_d_prices = calculate_prices([t_k_xauusd_d, t_k_p_xauusd_d, ch_p_xauusd_d, ch_p_c_xauusd_d[0], five_l_c_xauusd_d])

btcusd_h_prices = calculate_prices([t_k_btcusd_h, t_k_p_btcusd_h, ch_p_btcusd_h, ch_p_c_btcusd_h[0], five_l_c_btcusd_h])
ethusd_h_prices = calculate_prices([t_k_ethusd_h, t_k_p_ethusd_h, ch_p_ethusd_h, ch_p_c_ethusd_h[0], five_l_c_ethusd_h])
adausd_h_prices = calculate_prices([t_k_adausd_h, t_k_p_adausd_h, ch_p_adausd_h, ch_p_c_adausd_h[0], five_l_c_adausd_h])
btcusd_d_prices = calculate_prices([t_k_btcusd_d, t_k_p_btcusd_d, ch_p_btcusd_d, ch_p_c_btcusd_d[0], five_l_c_btcusd_d])
ethusd_d_prices = calculate_prices([t_k_ethusd_d, t_k_p_ethusd_d, ch_p_ethusd_d, ch_p_c_ethusd_d[0], five_l_c_ethusd_d])
adausd_d_prices = calculate_prices([t_k_adausd_d, t_k_p_adausd_d, ch_p_adausd_d, ch_p_c_adausd_d[0], five_l_c_adausd_d])

aapl_h_prices = calculate_prices([t_k_aapl_h, t_k_p_aapl_h, ch_p_aapl_h, ch_p_c_aapl_h[0], five_l_c_aapl_h])
nflx_h_prices = calculate_prices([t_k_nflx_h, t_k_p_nflx_h, ch_p_nflx_h, ch_p_c_nflx_h[0], five_l_c_nflx_h])
tsla_h_prices = calculate_prices([t_k_tsla_h, t_k_p_tsla_h, ch_p_tsla_h, ch_p_c_tsla_h[0], five_l_c_tsla_h])
aapl_d_prices = calculate_prices([t_k_aapl_d, t_k_p_aapl_d, ch_p_aapl_d, ch_p_c_aapl_d[0], five_l_c_aapl_d])
nflx_d_prices = calculate_prices([t_k_nflx_d, t_k_p_nflx_d, ch_p_nflx_d, ch_p_c_nflx_d[0], five_l_c_nflx_d])
tsla_d_prices = calculate_prices([t_k_tsla_d, t_k_p_tsla_d, ch_p_tsla_d, ch_p_c_tsla_d[0], five_l_c_tsla_d])

strategies = ['T_K', 'T_K_P', 'Ch_P', 'Ch_P_C', 'five_l_c']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=strategies,
    y=eurusd_h_prices,
    name='EURUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=gbpjpy_h_prices,
    name='GBPJPY',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=xauusd_h_prices,
    name='XAUUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=btcusd_h_prices,
    name='BTCUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=ethusd_h_prices,
    name='ETHUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=adausd_h_prices,
    name='ADAUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=aapl_h_prices,
    name='AAPL',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=nflx_h_prices,
    name='NFLX',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=tsla_h_prices,
    name='TSLA',
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0, title='1h Data', height=1000)
fig.show()

In [ ]:
strategies = ['T_K', 'T_K_P', 'Ch_P', 'Ch_P_C', 'five_l_c']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=strategies,
    y=eurusd_d_prices,
    name='EURUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=gbpjpy_d_prices,
    name='GBPJPY',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=xauusd_d_prices,
    name='XAUUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=btcusd_d_prices,
    name='BTCUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=ethusd_d_prices,
    name='ETHUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=adausd_d_prices,
    name='ADAUSD',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=aapl_d_prices,
    name='AAPL',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=nflx_d_prices,
    name='NFLX',
))

fig.add_trace(go.Bar(
    x=strategies,
    y=tsla_d_prices,
    name='TSLA',
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0, title='1d Data', height=1000)
fig.show()

In [ ]:
def calculate_timeline(strategies):
    
    result = []
    
    for strategy in strategies:
        strategy_array = [[], []]
        lastPrice = 0
        for row in strategy:
            if row['operation'] == 'sell':
                lastPrice += row['startValue'] - row['endValue']
                strategy_array[0].append(row['endTime'])
                strategy_array[1].append(lastPrice)
            else:
                lastPrice += row['endValue'] - row['startValue']
                strategy_array[0].append(row['endTime'])
                strategy_array[1].append(lastPrice)
        result.append(strategy_array)
    
    return result

eurusd_h_timeline = calculate_timeline([t_k_eurusd_h, t_k_p_eurusd_h, ch_p_eurusd_h, ch_p_c_eurusd_h[0], five_l_c_eurusd_h])
gbpjpy_h_timeline = calculate_timeline([t_k_gbpjpy_h, t_k_p_gbpjpy_h, ch_p_gbpjpy_h, ch_p_c_gbpjpy_h[0], five_l_c_gbpjpy_h])
xauusd_h_timeline = calculate_timeline([t_k_xauusd_h, t_k_p_xauusd_h, ch_p_xauusd_h, ch_p_c_xauusd_h[0], five_l_c_xauusd_h])
eurusd_d_timeline = calculate_timeline([t_k_eurusd_d, t_k_p_eurusd_d, ch_p_eurusd_d, ch_p_c_eurusd_d[0], five_l_c_eurusd_d])
gbpjpy_d_timeline = calculate_timeline([t_k_gbpjpy_d, t_k_p_gbpjpy_d, ch_p_gbpjpy_d, ch_p_c_gbpjpy_d[0], five_l_c_gbpjpy_d])
xauusd_d_timeline = calculate_timeline([t_k_xauusd_d, t_k_p_xauusd_d, ch_p_xauusd_d, ch_p_c_xauusd_d[0], five_l_c_xauusd_d])

btcusd_h_timeline = calculate_timeline([t_k_btcusd_h, t_k_p_btcusd_h, ch_p_btcusd_h, ch_p_c_btcusd_h[0], five_l_c_btcusd_h])
ethusd_h_timeline = calculate_timeline([t_k_ethusd_h, t_k_p_ethusd_h, ch_p_ethusd_h, ch_p_c_ethusd_h[0], five_l_c_ethusd_h])
adausd_h_timeline = calculate_timeline([t_k_adausd_h, t_k_p_adausd_h, ch_p_adausd_h, ch_p_c_adausd_h[0], five_l_c_adausd_h])
btcusd_d_timeline = calculate_timeline([t_k_btcusd_d, t_k_p_btcusd_d, ch_p_btcusd_d, ch_p_c_btcusd_d[0], five_l_c_btcusd_d])
ethusd_d_timeline = calculate_timeline([t_k_ethusd_d, t_k_p_ethusd_d, ch_p_ethusd_d, ch_p_c_ethusd_d[0], five_l_c_ethusd_d])
adausd_d_timeline = calculate_timeline([t_k_adausd_d, t_k_p_adausd_d, ch_p_adausd_d, ch_p_c_adausd_d[0], five_l_c_adausd_d])

aapl_h_timeline = calculate_timeline([t_k_aapl_h, t_k_p_aapl_h, ch_p_aapl_h, ch_p_c_aapl_h[0], five_l_c_aapl_h])
nflx_h_timeline = calculate_timeline([t_k_nflx_h, t_k_p_nflx_h, ch_p_nflx_h, ch_p_c_nflx_h[0], five_l_c_nflx_h])
tsla_h_timeline = calculate_timeline([t_k_tsla_h, t_k_p_tsla_h, ch_p_tsla_h, ch_p_c_tsla_h[0], five_l_c_tsla_h])
aapl_d_timeline = calculate_timeline([t_k_aapl_d, t_k_p_aapl_d, ch_p_aapl_d, ch_p_c_aapl_d[0], five_l_c_aapl_d])
nflx_d_timeline = calculate_timeline([t_k_nflx_d, t_k_p_nflx_d, ch_p_nflx_d, ch_p_c_nflx_d[0], five_l_c_nflx_d])
tsla_d_timeline = calculate_timeline([t_k_tsla_d, t_k_p_tsla_d, ch_p_tsla_d, ch_p_c_tsla_d[0], five_l_c_tsla_d])


def show_timeline_plot(timeline, title):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(x=timeline[0][0], y=timeline[0][1], name="T_K (" + str(len(timeline[0][1])) + ")")
    )

    fig.add_trace(
        go.Scatter(x=timeline[1][0], y=timeline[1][1], name="T_K_P (" + str(len(timeline[1][1])) + ")")
    )

    fig.add_trace(
        go.Scatter(x=timeline[2][0], y=timeline[2][1], name="Ch_P (" + str(len(timeline[2][1])) + ")")
    )

    fig.add_trace(
        go.Scatter(x=timeline[3][0], y=timeline[3][1], name="Ch_P_C (" + str(len(timeline[3][1])) + ")")
    )

    fig.add_trace(
        go.Scatter(x=timeline[4][0], y=timeline[4][1], name="Five_l_c (" + str(len(timeline[4][1])) + ")")
    )

    fig.update_layout(
        title_text=title,
    )
    
    fig.update_xaxes(constrain="range")

    fig.show()
    
show_timeline_plot(eurusd_h_timeline, "EURUSD 1h")
show_timeline_plot(eurusd_d_timeline, "EURUSD 1d")
show_timeline_plot(gbpjpy_h_timeline, "GBPJPY 1h")
show_timeline_plot(gbpjpy_d_timeline, "GBPJPY 1d")
show_timeline_plot(xauusd_h_timeline, "XAUUSD 1h")
show_timeline_plot(xauusd_d_timeline, "XAUUSD 1d")

show_timeline_plot(btcusd_h_timeline, "BTCUSD 1h")
show_timeline_plot(btcusd_d_timeline, "BTCUSD 1d")
show_timeline_plot(ethusd_h_timeline, "ETHUSD 1h")
show_timeline_plot(ethusd_d_timeline, "ETHUSD 1d")
show_timeline_plot(adausd_h_timeline, "ADAUSD 1h")
show_timeline_plot(adausd_d_timeline, "ADAUSD 1d")

show_timeline_plot(aapl_h_timeline, "AAPL 1h")
show_timeline_plot(aapl_d_timeline, "AAPL 1d")
show_timeline_plot(nflx_h_timeline, "NFLX 1h")
show_timeline_plot(nflx_d_timeline, "NFLX 1d")
show_timeline_plot(tsla_h_timeline, "TSLA 1h")
show_timeline_plot(tsla_d_timeline, "TSLA 1d")
    